### Import all needed libraries

In [34]:
#
# LIB IMPORTS
#
import os, sys
import pandas as pd
import numpy as np
from tqdm import tqdm

# for reading and displaying images
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

# torchvision for pre-trained models
from torchvision import datasets, transforms, models

# fastai
import fastai
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.imports import *


### Data Generation and Split
Using functions in external scripts, prepare the data and split

In [35]:
#
# GENERATE DATA
#
# Note this part may take some time
from dataload import *
from denoise import *
from normalize import *

csv_files = ['batch-data/batch1_results.csv',
             'batch-data/batch2_results.csv',
             'batch-data/batch3_results.csv',
             'batch-data/batch4_results.csv',
             'batch-data/batch5_results.csv',
             'batch-data/batch6_results.csv']

column_headers = ['AssignmentStatus','Answer.set_number','WorkerId','Answer.slider_values','Answer.slider_values2']

im_dict, data = make_predata(csv_files,column_headers)

# Denoise and get cleaned data
percentile = 90
cleaned_data = outlier_denoising(im_dict, data, percentile)

# X contains the image names
X, y1, y2 = format_data(cleaned_data)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-f90573f83795>", line 18, in <module>
    im_dict, data = make_predata(csv_files,column_headers)
  File "/Users/shwetha/UT/SD/MTurk_Bovik/Model/prelim-models/dataload.py", line 27, in make_predata
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 702, in parser_f
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 429, in _read
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 895, in __init__
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1122, in _make_engine
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1853, in __init__
  File "pandas/_libs/parsers.pyx", line 387, in pandas._libs.

OSError: Initializing from file failed

In [36]:
# Split the data into test and train set
test_size = 0.2
X_train, X_test, y1_train, y1_test = train_test_split(X, 
                                                      y1, 
                                                      test_size=test_size, 
                                                      random_state=123)
X_train, X_test, y2_train, y2_test = train_test_split(X, 
                                                      y2, 
                                                      test_size=test_size, 
                                                      random_state=123)


In [37]:
# NOTE: Beyond this point we are only trying slider 1

#
# Set up data format for regression based tl
#

import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('../data/8k_data/' + ID)
        y = self.labels[ID]

        return X, y
    
    
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    cudnn.benchmark = True

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

training_set = Dataset(X_train, y1_train)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(X_test, y1_test)
validation_generator = data.DataLoader(validation_set, **params)



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1c2170b8c8>
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 882, in _shutdown_workers
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'


In [38]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Decay LR by a factor of 0.1 every 7 epochs
from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-43abd5e6d0f3>", line 1, in <module>
    model = models.resnet18(pretrained=True)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torchvision/models/resnet.py", line 231, in resnet18
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torchvision/models/resnet.py", line 217, in _resnet
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/hub.py", line 499, in load_state_dict_from_url
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/serialization.py", line 419, in load
OSError: [Errno 24] Too many open files: '/Users/shwetha/.cache/torch/checkpoints/resnet18-5c106cde.pth'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-pa

OSError: [Errno 24] Too many open files: '/Users/shwetha/.cache/torch/checkpoints/resnet18-5c106cde.pth'

In [39]:
import time
import copy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)   
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
              
            running_loss = 0.0
            running_corrects = 0
            
            
            # Iterate over data.
            for inputs, labels in training_generator:
                print('here before')
                inputs = inputs.to(device)
                labels = labels.to(device)
                print('here after')
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
 

In [40]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=2)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Epoch 0/1
----------
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-33bc5605d8a7>", line 2, in <module>
    num_epochs=2)
  File "<ipython-input-39-0d56d3527263>", line 25, in train_model
    for inputs, labels in training_generator:
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 278, in __iter__
    return _MultiProcessingDataLoaderIter(self)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 647, in __init__
    self._worker_result_queue = multiprocessing_context.Queue()
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/context.py", line 102, in Queue
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/queues.py", line 41, in __init__
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/conne

OSError: [Errno 24] Too many open files